In [2]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [4]:
with open("data.txt", "r") as f:
    data = f.read()

In [7]:
chunks = data.split(">>>>")

In [8]:
chunks

['',
 '\nThe Original Report of the patient is : \n\n1IAN M KRONISH, MD, MPH \nAssistant Professor of Medicine \n \nCenter for Behavioral Cardiovascular \nHealth \n622 West 168th Street, PH9-311 \nNew York, NY 10032 \n212.342.1335 \n212.342.3441 Fax \nIk2293@columbia.edu \nwww.cumc.columbia.edu/cbch \n   February 10, 2016   Re: ABPM for John Doe, MRN: 1234567  Dear Dr. Jane Doe:  You referred your patient John Doe for ambulatory blood pressure monitoring (ABPM). Here is a \nbrief summary and interpretation of your patient’s results. A more detailed report is attached.   Reason for Referral: Suspected White-Coat Hypertension \nTotal Number of Blood Pressure Readings: 35 (minimum to be valid >10) % Valid Readings: 85% (optimal >80%)  Quality of Study: Adequate  \nSummary of Blood Pressure Readings:  Period Patient’s Blood Pressure Normal Blood Pressure \nDaytime (awake) 142/87 mmHg <135/85 mmHg \nNighttime (asleep) 125/75 mmHg <120/70 mmHg \n24-hour 135/85 mmHg <130/80 mmHg \n  Interpret

In [9]:
# if any element in this list is empty string, remove it 
chunks = [chunk for chunk in chunks if chunk != '']

In [10]:
chunks

['\nThe Original Report of the patient is : \n\n1IAN M KRONISH, MD, MPH \nAssistant Professor of Medicine \n \nCenter for Behavioral Cardiovascular \nHealth \n622 West 168th Street, PH9-311 \nNew York, NY 10032 \n212.342.1335 \n212.342.3441 Fax \nIk2293@columbia.edu \nwww.cumc.columbia.edu/cbch \n   February 10, 2016   Re: ABPM for John Doe, MRN: 1234567  Dear Dr. Jane Doe:  You referred your patient John Doe for ambulatory blood pressure monitoring (ABPM). Here is a \nbrief summary and interpretation of your patient’s results. A more detailed report is attached.   Reason for Referral: Suspected White-Coat Hypertension \nTotal Number of Blood Pressure Readings: 35 (minimum to be valid >10) % Valid Readings: 85% (optimal >80%)  Quality of Study: Adequate  \nSummary of Blood Pressure Readings:  Period Patient’s Blood Pressure Normal Blood Pressure \nDaytime (awake) 142/87 mmHg <135/85 mmHg \nNighttime (asleep) 125/75 mmHg <120/70 mmHg \n24-hour 135/85 mmHg <130/80 mmHg \n  Interpretation

In [11]:
# get word length of each chunk
chunks_word_length = [len(chunk.split()) for chunk in chunks]
chunks_word_length

[176, 237, 40, 444, 525, 127]

In [12]:
from langchain_openai import OpenAIEmbeddings

In [13]:
em_model = OpenAIEmbeddings(model = "text-embedding-3-small")

In [14]:
embeddings = em_model.embed_documents(chunks)

In [17]:
len(embeddings), len(embeddings[0])

(6, 1536)

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(em_model)

In [24]:
from langchain_core.documents import Document   

In [25]:
docs = [Document(page_content=chunk) for chunk in chunks]

In [26]:
docs

[Document(metadata={}, page_content='\nThe Original Report of the patient is : \n\n1IAN M KRONISH, MD, MPH \nAssistant Professor of Medicine \n \nCenter for Behavioral Cardiovascular \nHealth \n622 West 168th Street, PH9-311 \nNew York, NY 10032 \n212.342.1335 \n212.342.3441 Fax \nIk2293@columbia.edu \nwww.cumc.columbia.edu/cbch \n   February 10, 2016   Re: ABPM for John Doe, MRN: 1234567  Dear Dr. Jane Doe:  You referred your patient John Doe for ambulatory blood pressure monitoring (ABPM). Here is a \nbrief summary and interpretation of your patient’s results. A more detailed report is attached.   Reason for Referral: Suspected White-Coat Hypertension \nTotal Number of Blood Pressure Readings: 35 (minimum to be valid >10) % Valid Readings: 85% (optimal >80%)  Quality of Study: Adequate  \nSummary of Blood Pressure Readings:  Period Patient’s Blood Pressure Normal Blood Pressure \nDaytime (awake) 142/87 mmHg <135/85 mmHg \nNighttime (asleep) 125/75 mmHg <120/70 mmHg \n24-hour 135/85 m

In [27]:
store = vector_store.from_documents(docs, em_model)

In [28]:
retriever = store.as_retriever(
    search_kwargs = {"k": 2}
)

In [49]:
res = retriever.invoke(
    "What are some alternative medicines for paracetamol?"
)

In [50]:
res

[Document(id='d06031e8-243a-4cea-8ec2-8b0a37deaaa8', metadata={}, page_content='\nAll information about paracetamol | \n{\n    "medication_name": "Paracetamol",\n    "generic_name": "paracetamol",\n    "ingredients": [\n        "paracetamol"\n    ],\n    "brand_names": [\n        "Panadol",\n        "Calpol",\n        "Tylenol",\n        "Alvedon"\n    ],\n    "medicine_use": "used to temporarily relieve mild-to-moderate pain and fever",\n    "common_side_effects": [\n        "Redness or soreness in or around the rectum"\n    ],\n    "severe_side_effects": [\n        "Allergic reactions, skin rashes, itching or hives, swelling of the throat, tongue or face, shortness of breath or wheezing",\n        "Skin rash or peeling, or mouth ulcers",\n        "Breathing problems",\n        "Unexplained bruising or bleeding, becoming unusually tired, getting more infections than usual",\n        "Liver problems, nausea, sudden weight loss, loss of appetite, yellowing of the eyes and skin"\n    ],\

In [61]:
relevant_doc = res[0].page_content +  "\n" + res[1].page_content

In [62]:
relevant_doc

'\nAll information about paracetamol | \n{\n    "medication_name": "Paracetamol",\n    "generic_name": "paracetamol",\n    "ingredients": [\n        "paracetamol"\n    ],\n    "brand_names": [\n        "Panadol",\n        "Calpol",\n        "Tylenol",\n        "Alvedon"\n    ],\n    "medicine_use": "used to temporarily relieve mild-to-moderate pain and fever",\n    "common_side_effects": [\n        "Redness or soreness in or around the rectum"\n    ],\n    "severe_side_effects": [\n        "Allergic reactions, skin rashes, itching or hives, swelling of the throat, tongue or face, shortness of breath or wheezing",\n        "Skin rash or peeling, or mouth ulcers",\n        "Breathing problems",\n        "Unexplained bruising or bleeding, becoming unusually tired, getting more infections than usual",\n        "Liver problems, nausea, sudden weight loss, loss of appetite, yellowing of the eyes and skin"\n    ],\n    "immediate_attention_side_effects": [\n        "paleness",\n        "nause

In [52]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os 

llm = ChatOpenAI(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    openai_api_key=os.getenv("OPENAI_API_KEY")
)


C:\Users\91790\AppData\Local\Temp\ipykernel_18468\291976851.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [53]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)


In [55]:
response = qa_chain.run("Show me side effects of ibuprofen")

Common side effects of Ibuprofen include:
- Nausea
- Vomiting
- Gas
- Bleeding
- Dizziness
- Headache

Severe side effects of Ibuprofen can include:
- Stomach or intestinal bleeding
- Heart attack
- Stroke
- Liver problems
- Kidney problems
- Allergic reaction
- Severe skin reaction
- Vision changes
- Anemia

Immediate attention side effects of Ibuprofen include:
- Signs of an allergic reaction to ibuprofen
- Signs of a heart attack or stroke
- Signs of stomach bleeding
- Liver problems
- Kidney problems

In [2]:
from openai import OpenAI
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [4]:
from pydantic import BaseModel

In [5]:
class Query(BaseModel): 
    is_followup: bool 
    requires_new_context: bool

In [6]:
prompt = f"""
    Analyze this query in the context of the current conversation.
    
    Current topic: is everything ok with my report ? 
    
    Latest conversation:
    Yes, the report is fine.

    New query: What are some alternative medicines for paracetamol?
    
    Determine:
    1. Is this a follow-up to the previous conversation?
    2. Does it require fetching new documentation or can it be answered with existing context?
"""
classification = client.beta.chat.completions.parse(
    model = "gpt-4o-mini",
    messages = [
        {
            "role": "user", 
            "content": prompt
        }
    ],
    response_format = Query
) 


In [19]:
classification.choices[0].message.content

import json 
json.loads(classification.choices[0].message.content).get("is_followup")

False